In [86]:
import os 
import numpy as np 
import pandas as pd
from collections import defaultdict
import itertools
import ast

In [87]:
small_title_set = {"The Hunger Games (The Hunger Games, #1)", "Harry Potter and the Sorcerer's Stone (Harry Potter, #1)","Twilight (Twilight, #1)", "To Kill a Mockingbird", "The Great Gatsby", "The Fault in Our Stars", "The Hobbit", "The Catcher in the Rye", "Angels & Demons  (Robert Langdon, #1)", "Pride and Prejudice"}
print(len(small_title_set))

10


In [88]:
#hyper parameters
rating_threshold = 3.5 #only counted the usr rating if it is rated above the rating threshold
user_count_threshold = 10 #only keep the bk_id if the bk is rated by user for more than user_count_threshold times
split = [0.8, 0.1, 0.1] #train, dev, test split
#INPUT_DIR = '../../../data/the-movies-dataset/'
INPUT_DIR ='../../data/goodbooks-10k-master/'
OUTPUT_DIR = '../box-code/data/book_data/big/small/'

In [89]:
#read in bk_id and usr rating
ratings_file = INPUT_DIR + 'ratings.csv'
df = pd.read_csv(ratings_file, delimiter=',')
df.dataframeName = 'ratings.csv'
print(df.head(5))

   user_id  book_id  rating
0        1      258       5
1        2     4081       4
2        2      260       5
3        2     9296       5
4        2     2318       3


In [90]:
#read in all tag name and tag_id
tag_name_file = INPUT_DIR + 'tags.csv'
df_tag_name = pd.read_csv(tag_name_file, delimiter=',')
df_tag_name.dataframeName = 'tags.csv'
#print(df_tag_name.size)
print(df_tag_name.head(5))

   tag_id tag_name
0       0        -
1       1     --1-
2       2    --10-
3       3    --12-
4       4   --122-


In [91]:
#read in goodreads_book_id and their labeled tags with tag_id
bk_tag_file = INPUT_DIR + 'book_tags.csv'
df_bk_tag = pd.read_csv(bk_tag_file, delimiter=',')
df_bk_tag.dataframeName = 'book_tags.csv'
#df_bk_tag.drop(columns=['count']) drop count
print(df_bk_tag.head(5))

   goodreads_book_id  tag_id   count
0                  1   30574  167697
1                  1   11305   37174
2                  1   11557   34173
3                  1    8717   12986
4                  1   33114   12716


In [92]:
#read in valid tag names
genre_name_file = INPUT_DIR + "book-genres.txt"
df_valid_tag = pd.read_csv(genre_name_file, sep= "\n")
df_valid_tag.dataframeName = 'book-genres.txt'
df_valid_tag = df_valid_tag[~df_valid_tag.genres.str.contains(" ")]#delete the lines that counts how many books has the tag
print(df_valid_tag.size)

1232


In [93]:
#number of tags obtained from website: 1232
#number of tags in the tag id vs name datafile:  34252
#overlap of the two: 833

In [94]:
#read in bk id and goodreads book id, later convert all goodreads bk id to bk id
bk_id_file = INPUT_DIR + 'books.csv'
df_bk_id = pd.read_csv(bk_id_file, delimiter=',')
df_bk_id.dataframeName = 'books.csv'
#print(df_bk_id.head(10))
df_bk_id=df_bk_id[['book_id','goodreads_book_id', 'title']]

In [95]:
#drop unwanted data in the metadata 
print(len(df_bk_id))
df_bk_id = df_bk_id[df_bk_id['title'].isin(small_title_set)]
print(len(df_bk_id))

10000
10


In [84]:
#create a dict map goodreads bk id to bk id, we will use bk id to train
id_name_dict={} #key: good_read_id, val: bk_id
small_book_id_set = set()
small_gd_read_id_set = set()
title_bk_id_dict = {} #key bk_id val: bk title
for index, row in df_bk_id.iterrows():
    id_name_dict[row['goodreads_book_id']] = row['book_id'] 
    small_book_id_set.add(row['book_id'] )
    small_gd_read_id_set.add(row['goodreads_book_id'])
    title_bk_id_dict[row['book_id']] = row['title']

In [68]:
print(small_book_id_set)
print(small_gd_read_id_set)

{2, 3, 4, 8354, 6, 7047, 8, 4234}
{2657, 3, 11870085, 32453, 41865, 252938, 5107, 93724}


In [85]:
for k, v in title_bk_id_dict.items():
    print(k, v)

2 Harry Potter and the Philosopher's Stone
3 Twilight
4 To Kill a Mockingbird
6 The Fault in Our Stars
8 The Catcher in the Rye
4234 Twilight
7047 Twilight
8354 Twilight


In [69]:
valid_gen = set([])#set of names of valid tags, this list is obtained from the goodreads website
for index, row in df_valid_tag.iterrows():
    valid_gen.add(row['genres'])
print(len(valid_gen))

1232


In [70]:
#create dictionary of valid tag id and tag names, key: valid tag id, value: valid tag name
valid_tag_dict = {}
for index, row in df_tag_name.iterrows():
    tag_name = row['tag_name']
    tag_id = row['tag_id']
    if tag_name in valid_gen:
        valid_tag_dict[tag_id] = tag_name 
print(len(valid_tag_dict.keys()))

833


In [71]:
#drop unwanted data in the rating data
print(len(df))
df = df[df['book_id'].isin(small_book_id_set)]
print(len(df))

5976479
84331


In [72]:
#drop unwanted data in the rating data
print(len(df_bk_tag))
df_bk_tag = df_bk_tag[df_bk_tag['goodreads_book_id'].isin(small_gd_read_id_set)]
print(len(df_bk_tag))

999912
800


In [73]:
#preprocess usr rating data, prepare for bk bk pairs
df = df[df.rating >= rating_threshold]#Drop all entry with rating lower than the threshold
bk_usr_dict = defaultdict(list)#dict with key: bk_id, val: the list of user_ids rated the bk above the rating threshold
for index, row in df.iterrows():    
    movieId = int(row['book_id'])
    usrId = int(row['user_id'])
    bk_usr_dict[movieId].append(usrId)

for key in list(bk_usr_dict.keys()):#Drop all movie entry if number of rates below the user count threshold 
    if len(bk_usr_dict[key]) < user_count_threshold:
        del bk_usr_dict[key]
print(len(bk_usr_dict.keys()))#736 for 4.5 500

8


In [74]:
#create dict of each book and the tags they are labeled with, key: bk_id, val: list of tag names
bk_tag_dict = defaultdict(list)
valid_data_count = 0## count the number of pieces of valid bk-tag relations 
for index, row in df_bk_tag.iterrows():
    gd_read_id = row['goodreads_book_id']
    tag = row['tag_id']
    bk_id =id_name_dict[gd_read_id]
    #check if the tag is a valid tag, and the bk is valid after thresholding and filtering
    if (tag in valid_tag_dict) and (bk_id in bk_usr_dict):
        bk_tag_dict[bk_id].append(valid_tag_dict[tag])
        valid_data_count += 1##
        
#there are totally 99912 rows in the bk-tag datafile, valid_data_count is the number of rows we use    
print(valid_data_count)

169


In [75]:
print(len(bk_tag_dict.keys()))
print(len(bk_usr_dict.keys()))

8
8


In [76]:
def marginal_prob(movie_id, count_matrix, num_users):
    '''function to get the marginal prob:
       P(movie_id1)       
    '''
    margn_count = count_matrix[ movie_id]
    return margn_count/num_users

def joint_prob(movie_id1, movie_id2, count_matrix, num_users):
    '''function to get the joint prob:
        P(movie_id1, movie_id2)
    '''
    key = (movie_id1, movie_id2) if movie_id1<= movie_id2 else (movie_id2, movie_id1) 
    joint_count = count_matrix[key] 
    return joint_count/num_users

def conditional_prob(movie_id1, movie_id2, count_matrix, marginal_matrix):
    '''function to get the conditional prob:
        P(movie_id1 | movie_id2)       
    '''
    key = (movie_id1, movie_id2) if movie_id1<= movie_id2 else (movie_id2, movie_id1)
    joint_count = count_matrix[key]
    if joint_count == 0:
        return 0
    margn_count = marginal_matrix[movie_id2]
    '''
   
        print("------")
        print("id1: ", movie_id1, "id2: ", movie_id2)
        print(count_matrix[(movie_id1, movie_id2)])
        print(marginal_matrix[movie_id1], marginal_matrix[movie_id2])
        print(joint_count,margn_count,  joint_count/margn_count)
        print("------")
   '''
    return joint_count/margn_count

#given ordered pair of key(a,b), generate both p(b|a) and p(a|b)
def data_generation(a,b, count_matrix, marginal_matrix):
    p_a_b = conditional_prob(a, b, count_matrix, marginal_matrix)#p(a|b)
    p_b_a = conditional_prob(b, a, count_matrix, marginal_matrix)#p(b|a)
    return(((a,b),p_b_a), ((b,a),p_a_b))

In [77]:
#create the movie_genre_master file
file = open(OUTPUT_DIR+"book_genre_master.txt", "w") 
for movie_id in list(bk_tag_dict.keys()):
    for genre_id in bk_tag_dict[movie_id]:
        file.write("IsA\t"+str(movie_id)+"\t" + str(genre_id) + "\t" + str(1)+"\n")
file.close() 

In [78]:
#usr_bk_dict: dict with key: user, val: the book_id this usr rated above given threshold
usr_bk_dict = defaultdict(list)
for key, val in bk_usr_dict.items():
    for i in val:
        usr_bk_dict[i].append(key)
        
#to make sure if a user rate a movie, the user only rate the movie once
for key, val in usr_bk_dict.items():
    usr_bk_dict[key] = list(set(val))

#movie_movie_count: key: (bk_id_i, bk_id_j) (bk_id_i != bk_id_j and bk_id_i < bk_id_j) 
#                 val: co-ocurrence count of bk_id_i and bk_id_j
#movie_movie_count_marginal: key: (bk_id_i) val: ocurrence count of bk_id_i in the dataset
movie_movie_count =  defaultdict(lambda: 0)
movie_movie_count_marginal = defaultdict(lambda:0)
for key, val in usr_bk_dict.items():
    for i in range(len(val)):
        movie_movie_count_marginal[val[i]] += 1 #marginal count
        for j in range(i+1, len(val)):#marginal count not included in book_book_count
            movie_movie_count[(val[i], val[j]) if val[i] <= val[j] else (val[j], val[i])] += 1
            

In [79]:
gen_gen_count =  defaultdict(lambda: 0)
gen_gen_count_marginal = defaultdict(lambda:0)

for key, val in bk_tag_dict.items():
     for i in range(len(val)):
        gen_gen_count_marginal[val[i]] += 1 #marginal count
        for j in range(i+1, len(val)):#marginal count not included in book_book_count
            gen_gen_count[(val[i], val[j]) if val[i] <= val[j] else (val[j], val[i])] += 1

In [80]:
#create the movie vocab file
file = open(OUTPUT_DIR+"book_vocabulary.txt", "w") 
marginal_keys = list(movie_movie_count_marginal.keys())
for i in marginal_keys:
    file.write(str(i)+"\n") 
file.close() 

#create the movie marginal prob file, order of value match order of books id in vocab file
file = open(OUTPUT_DIR+"book_marginal_prob.txt", "w") 
N = len(usr_bk_dict)#number of user
for i in marginal_keys:
    file.write(str(marginal_prob(i, movie_movie_count_marginal, N))+"\n")
file.close() 

In [81]:
#create the genre vocab file
file = open(OUTPUT_DIR+"genre_vocabulary.txt", "w") 
marginal_keys = list(gen_gen_count_marginal.keys())
for i in marginal_keys:
    file.write(str(i)+"\n") 
file.close() 

#create the genre marginal prob file, order of value match order of books id in vocab file
file = open(OUTPUT_DIR+"genre_marginal_prob.txt", "w") 
N = len(bk_tag_dict)#number of total movies
for i in marginal_keys:
    file.write(str(marginal_prob(i, gen_gen_count_marginal, N))+"\n")
file.close() 

In [82]:
temp = list(movie_movie_count.items())
movie_movie_master_data=[]
for data in temp:
    a,b = data_generation(data[0][0],data[0][1], movie_movie_count, movie_movie_count_marginal)
    movie_movie_master_data.append(a)
    movie_movie_master_data.append(b)
#create the train data file
file = open(OUTPUT_DIR + "book_book_master.txt", "w") 
for i in movie_movie_master_data:
    if str(i[1])!="0":
        file.write("IsA\t"+str(i[0][0])+"\t" + str(i[0][1]) + "\t" + str(i[1])+"\n")
file.close() 

In [83]:
temp = list(gen_gen_count.items())
gen_gen_master_data=[]
for data in temp:
    a,b = data_generation(data[0][0],data[0][1], gen_gen_count, gen_gen_count_marginal)
    gen_gen_master_data.append(a)
    gen_gen_master_data.append(b)
#create the train data file
file = open(OUTPUT_DIR + "genre_genre_master.txt", "w") 
for i in gen_gen_master_data:
    if str(i[1])!="0":
        file.write("IsA\t"+str(i[0][0])+"\t" + str(i[0][1]) + "\t" + str(i[1])+"\n")
file.close() 